# Simple Plotter

In [1]:
import os, multiprocessing, sys
import copy
import math
from array import array
from ROOT import ROOT, gROOT, gStyle, gRandom, TSystemDirectory
from ROOT import TFile, TChain, TTree, TCut, TH1, TH1F, TH2F, THStack, TGraph, TGraphAsymmErrors
from ROOT import TStyle, TCanvas, TPad
from ROOT import TLegend, TLatex, TText, TLine, TBox
import time

cwd=os.getcwd()
sys.path.append(cwd+"/Utils/")

from drawLambda import *
from variables import *
from selections_SSLep import *
from samplesVH import *

gROOT.SetBatch(True)
gStyle.SetOptStat(0)
gROOT.Macro('functions.C')
%jsroot

Welcome to JupyROOT 6.12/06


# Environment Path and Options

In [2]:
#NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/datav7-skim-MC/"
NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/datav8-skim/" ##Zmass decent agreement but low event at tail
#NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/VHv9-skim_reprocess_v1/" ##Zmass BAD
#NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/dataset-v09-VH-reprocess/"
#NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/dataset-v10-VH/"
#NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/dataset-v11-VH/"

LUMI        = 35800. # pb-1                                                                                                                
data = ["data_obs"]
#back = [ "ttV", "WJetsToLNu", "TTbar-SL", "WW", "WZ", "ST", "TTbar-DiLep", "DYJetsToLL_Pt" ]   #DYJetsToLL                                              
back = [ "ttV" , "WW", "WZ", "TTbar-SL", "ST", "TTbar-DiLep", "WJetsToLNu", "DYJetsToLL" ] #WJetsToLNu, "VVV"
sign=[]
BLIND=False
POISSON= False
SIGNAL      = 1.
RATIO       = 4
jobs        = []

# Selection option

In [3]:
import pandas as pd
from collections import OrderedDict
from IPython.display import display, HTML
pd.set_option('display.max_colwidth',10000)
ALLTABLE={}
TABLE=[]
#Selection=['OSSF','OSOF','SSSF','SSOF']
for sel in selection:
    TABLE.append(OrderedDict({
        'Selection Region' : sel,
        'Description' : selection[sel]
    }))

ALLTABLE=pd.DataFrame(TABLE)
print "CHOOSE SELECTION"
display(ALLTABLE)

CHOOSE SELECTION


,Selection Region,Description
0,OSemu,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( (Muon_charge[0]!=Electron_charge[0]) && Muon_pt[0]>27 && Electron_pt[0]>25 && Muon_pfRelIso03_all[0]<0.1 && Muon_mediumId[0]>0 && Electron_cutBased[0]>0 && Electron_pfRelIso03_all[0]<0.1 )) ))
1,SSmumu2b,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( (Muon_charge[0]==Muon_charge[1]) && Muon_pt[0]>25 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.1 && Jet_btagCSVV2[1]>0.8 )) ))
2,Trigger,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( 1==1 ))
3,OSmumu,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( (Muon_charge[0]!=Muon_charge[1]) && Muon_pt[0]>30 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.15 && Muon_pfRelIso03_all[1]<0.15 )) ))
4,SSmumu,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( (Muon_charge[0]==Muon_charge[1]) && Muon_pt[0]>25 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.1 )) ))
5,SSmumu1b,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( (Muon_charge[0]==Muon_charge[1]) && Muon_pt[0]>25 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.1 && Jet_btagCSVV2[0]>0.8 )) ))
6,DiLepTT,"(( (( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( Muon_pt[0]>24 && Electron_pt[0]>20 )) )) ) && ( invariantMass(Electron_pt[0], Electron_eta[0], Electron_phi[0], Electron_mass[0], Muon_pt[0], Muon_eta[0], Muon_phi[0], Muon_mass[0])>15 ))"
7,OSee,(( (HLT_Ele25_WPTight_Gsf || HLT_Ele27_eta2p1_WPLoose_Gsf || HLT_Ele105_CaloIdVT_GsfTrkIdT) ) && ( (( 1==1 ) && ( (Electron_charge[0]!=Electron_charge[1]) && Electron_pt[0]>25 && Electron_pt[1]>17 && Electron_pfRelIso03_all[0]<0.1 && Electron_pfRelIso03_all[1]<0.1 )) ))


In [4]:
#var="MHTju_pt"
#var="Jet_jetId[0]"
var="Zmass"
#var="nJet"
#var="nElectron"
cr="OSmumu"
#cr="Trigger"
primaryDataset = getPrimaryDataset(selection[cr])
print "Selected", selection[cr]

Selected (( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( 1==1 ) && ( (Muon_charge[0]!=Muon_charge[1]) && Muon_pt[0]>30 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.15 && Muon_pfRelIso03_all[1]<0.15 )) ))


In [5]:
if not BLIND:
    processes=data+back
elif BLIND:
    processes=back

if not sign:
    Histlist=ProjectDraw(var, cr, LUMI, processes, primaryDataset, NTUPLEDIR)
else:
    Histlist=ProjectDraw(var, cr, LUMI, processes, primaryDataset, NTUPLEDIR)

Zmass : Inv( Muon^(+/-) , Muon^(-/+) )
VAR =  invariantMass(Muon_pt[0], Muon_eta[0], Muon_phi[0], Muon_mass[0], Muon_pt[1], Muon_eta[1], Muon_phi[1], Muon_mass[1])
DATA, NO HLT :  SingleMuonRun2016B-03Feb2017_ver1-v1
DATA, NO HLT :  SingleMuonRun2016B-03Feb2017_ver2-v2
DATA, NO HLT :  SingleMuonRun2016C-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016D-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016E-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016F-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016G-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016H-03Feb2017_ver2-v1
DATA, NO HLT :  SingleMuonRun2016H-03Feb2017_ver3-v1
MC, With HLT :  TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8_ext1-v3
MC, With HLT :  TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8_ext1-v1
MC, With HLT :  ttHToNonbb_M125_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8-v1
MC, With HLT :  WpWpJJ_13TeV-powheg-pythia8_TuneCUETP8M1-v1
MC, With HLT :  WmWmJJ_13TeV-powheg-pythia8_TuneCUETP8M1-v1
MC, With HLT :  WZTo3

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [6]:
if len(back)>0:
    #If data_obs present, dummy BkgSum == first background process                                                                        \
                                                                                                                                           
    Histlist['BkgSum'] = Histlist['data_obs'].Clone("BkgSum") if 'data_obs' in Histlist else Histlist[back[0]].Clone("BkgSum")
    Histlist['BkgSum'].Reset("MICES")
    Histlist['BkgSum'].SetFillStyle(3003)
    Histlist['BkgSum'].SetFillColor(1)
    for i, s in enumerate(back): Histlist['BkgSum'].Add(Histlist[s])

if len(back)==0 and len(data)==0:
    for i, s in enumerate(sign):
        Histlist[s].Scale(1./Histlist[s].Integral())
        Histlist[s].SetFillStyle(0)

In [7]:
if len(data+back)>0:
    out = draw(Histlist, data if not BLIND else [], back, sign, SIGNAL, RATIO, POISSON, variable[var]['log'])
    out[0].cd(1)
    drawCMS(LUMI, "Preliminary")
    out[0].Update()
    printTable(Histlist, sign)
else:
    out = drawSignal(Histlist, sign, variable[var]['log'])
    out[0].cd(1)
    drawCMS(LUMI, "Simulation")
    out[0].Update()

  Ratio:	1.049 +- 0.001
Sample                  Events          Entries         %
--------------------------------------------------------------------------------
data_obs             	3659021.00 	3659021    	104.87     %
--------------------------------------------------------------------------------
TTbar-DiLep          	114807.93  	820263     	3.29       %
WJetsToLNu           	28379.27   	283        	0.81       %
TTbar-SL             	3564.13    	27031      	0.10       %
ST                   	899.58     	6910       	0.03       %
WW                   	0.29       	58         	0.00       %
ttV                  	667.17     	110504     	0.02       %
DYJetsToLL           	3340012.68 	751016     	95.73      %
WZ                   	705.38     	53232      	0.02       %
--------------------------------------------------------------------------------
BkgSum               	3489036.43 	1769297    	100.00     %
--------------------------------------------------------------------------------
----

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [8]:
out[0].Draw()

## NOTES:

### Physics Object:

- Jet: cleaning from muon and electron:
    - Jet:
        - puID>4
        - pt>30
        - |eta|<2.5
    - Muon:
        - MediumID
        - pt>5
        - DeltaR>0.4
    - Electron:
        - Cutbase>4
        - pt>15
        - DeltaR>0.4
        
        
        

- OSmumu -> With jet cleaning and kinematics requirement on the jet, quality cut on muon, Zmass look stable, but 30% more data

- SSmumu -> MHTju_pt = data (180057) ; BkgSum (233304) = Data more ; nJet = data (180057) ; BkgSum (233281)

In [9]:
### TOp up WJet HT and DY PT